In [51]:
import numpy as np
from sklearn.cluster import DBSCAN
import pandas as pd

In [2]:
filePath = r"C:\Users\ashamail\Box\Shared Materials_Abdullah_Hasan\Abdullah Data\ring_centers.npy"
data = np.load(filePath)
# data = data[:2000]

data1 = list()
for x in range(236):
    
    data1.append(data[:,x,0:3].tolist())

data = data1
# data.shape

In [3]:
# data = np.load(filePath)
# data[:,x,0:3].tolist()

In [84]:
# data

In [5]:
# data = np.load(filePath)
# data[:2].shape

In [6]:
# data

In [7]:
# data1

In [8]:
# np.reshape(data[:2], (236, 2, 3))a

In [9]:
# data

In [10]:
class ConvoyCandidate(object):
    """
    Attributes:
        indices(set): The object indices assigned to the convoy
        is_assigned (bool):
        start_time (int):  The start index of the convoy
        end_time (int):  The last index of the convoy
    """
    __slots__ = ('indices', 'is_assigned', 'start_time', 'end_time')

    def __init__(self, indices, is_assigned, start_time, end_time):
        self.indices = indices
        self.is_assigned = is_assigned
        self.start_time = start_time
        self.end_time = end_time

    def __repr__(self):
        return '<%r %r indices=%r, is_assigned=%r, start_time=%r, end_time=%r>' % (self.__class__.__name__, id(self), self.indices, self.is_assigned, self.start_time, self.end_time)

In [11]:
class CMC(object):
    """Coherence Moving Cluster (CMC) algorithm

    Attributes:
        k (int):  Min number of consecutive timestamps to be considered a convoy
        m (int):  Min number of elements to be considered a convoy
    """
    def __init__(self, clf, k, m):
        self.clf = clf
        self.k = k
        self.m = m

    def fit_predict(self, X, y=None, sample_weight=None):
        convoy_candidates = set()
        columns = len(X[0])
        column_iterator = range(columns)
        output_convoys = []

        for column in column_iterator:
            current_convoy_candidates = set()
            values = [row[column] if isinstance(row[column], (list, set)) else [row[column]] for row in X]
            if len(values) < self.m:
                continue
            clusters = self.clf.fit_predict(values, y=y, sample_weight=sample_weight)
            unique_clusters = set(clusters)
            clusters_indices = dict((cluster, ConvoyCandidate(indices=set(), is_assigned=False, start_time=None, end_time=None)) for cluster in unique_clusters)

            for index, cluster_assignment in enumerate(clusters):
                clusters_indices[cluster_assignment].indices.add(index)

            # update existing convoys
            for convoy_candidate in convoy_candidates:
                convoy_candidate_indices = convoy_candidate.indices
                convoy_candidate.is_assigned = False
                for cluster in unique_clusters:
                    cluster_indices = clusters_indices[cluster].indices
                    cluster_candidate_intersection = cluster_indices & convoy_candidate_indices
                    if len(cluster_candidate_intersection) < self.m:
                        continue
                    convoy_candidate.indices = cluster_candidate_intersection
                    current_convoy_candidates.add(convoy_candidate)
                    convoy_candidate.end_time = column
                    clusters_indices[cluster].is_assigned = convoy_candidate.is_assigned = True

                # check if candidates qualify as convoys
                candidate_life_time = (convoy_candidate.end_time - convoy_candidate.start_time) + 1
                if (not convoy_candidate.is_assigned or column == column_iterator[-1]) and candidate_life_time >= self.k:
                    output_convoys.append(convoy_candidate)

            # create new candidates
            for cluster in unique_clusters:
                cluster_data = clusters_indices[cluster]
                if cluster_data.is_assigned:
                    continue
                cluster_data.start_time = cluster_data.end_time = column
                current_convoy_candidates.add(cluster_data)
            convoy_candidates = current_convoy_candidates
        return output_convoys

In [42]:
# clustering = DBSCAN(eps=2,min_samples=2).fit_predict(test_data)
# print(clustering)

data = ([[1,1],[25,4],[54,3]],
        [[3,1],[4,2],[5,1]],
        [[3,2],[6,2],[5,3]],
        [[3,3],[5,2],[5,3]],
        [[51,1],[51,2],[56,3]])

# data = ([[936], [1870], [2814], [3719], [4601]],
#              [[939], [1873], [2817], [3750], [4525]],
#              [[943], [1877], [2825], [3741], [4236]],
#              [[946], [1879], [2826], [3762], [4947]])


In [35]:
# Clustering using DBSCAN
clustering_clf = DBSCAN(eps=2, min_samples=2)

In [36]:
# data = data.tolist()

In [37]:
# data.shape

In [38]:
# # # # Test data of 3D Coordinates
# # # # Elements (Molecules) are in row, timesteps are in column

# data = ([[3,1,3],[3,2,4],[2,4,5]],
#         [[4,2,2],[6,2,3],[4,4,5]],
#         [[1,1,3],[51,2,-1],[55,3,0]],
#         [[51,2,1],[52,2,4],[56,-3, 2]])

In [43]:
# Min elements for convoy = m
# Min consecutive timesteps = k

clf = CMC(clustering_clf, k=3, m=2)

# Convoy calculation using Test data
convoys = clf.fit_predict(data)
file = open("output.txt", "w")
for convoy in convoys:
    print('Detected Convoy')
#     for i in convoy.indices:
#         print('%i: %r - Start Time: %r, End Time: %r' % (i, data[i], convoy.start_time, convoy.end_time))
    print(convoy)
#     file.write('%r - S: %r, E: %r\n' % (convoy.indices, convoy.start_time, convoy.end_time))
#     file.write(str(convoy.start_time))
#     file.write(str(convoy.end_time))
# file.close()

Detected Convoy
<'ConvoyCandidate' 1413172163776 indices={1, 2, 3}, is_assigned=True, start_time=0, end_time=2>


In [44]:
data = np.load(filePath)

In [81]:
df1 = pd.DataFrame()
for x in range(236):
    df = pd.DataFrame(data[:50,x,:])
    df["obj_id"] = x
    df["t"] = range(50)
    df = df.rename(columns={0: "x", 1: "y", 2: "z"})
    df = df[["obj_id", "t", "x", "y", "z"]]
    df1 = pd.concat([df1, df])

df1 = df1.reset_index(drop=True)

In [82]:
df1

,obj_id,t,x,y,z
0,0,0,13.026784,-0.772840,31.035849
1,0,1,12.945460,-0.664879,31.586801
2,0,2,13.274155,-0.978428,30.889317
3,0,3,13.115873,-0.843575,31.102140
4,0,4,12.739020,-0.834161,31.084617
...,...,...,...,...,...
11795,235,45,53.685381,40.766819,69.629093
11796,235,46,52.938960,41.627408,69.077421
11797,235,47,52.929715,41.393910,68.160006
11798,235,48,52.930080,41.892993,68.709482


In [83]:
df1.to_csv(r"C:\Users\ashamail\Desktop\Coherent-Moving-Cluster\PythonSource\resource\data2-50.csv", index=False)

In [79]:
df1[:11800]

,obj_id,t,x,y,z
0,0,0,13.026784,-0.772840,31.035849
1,0,1,12.945460,-0.664879,31.586801
2,0,2,13.274155,-0.978428,30.889317
3,0,3,13.115873,-0.843575,31.102140
4,0,4,12.739020,-0.834161,31.084617
...,...,...,...,...,...
11795,0,11795,12.497712,-0.957486,31.512845
11796,0,11796,12.560881,-0.382503,32.018365
11797,0,11797,12.635814,-0.364476,31.865283
11798,0,11798,12.611012,-0.364559,31.787725
